## 종만북 chapter 4. 알고리즘 시간 복잡도 분석

반복문이 수행시간을 지배한다.

### 선형 시간 알고리즘
수행시간이 입력 크기에 정비례.
### 선형 이하 시간 알고리즘
입력으로 주어진 자료에 대해 미리 알고있는 지식이 있다면(정렬되었다) 가능. 이진탐색(binary search)

### 지수 시간 알고리즘
#### 다항 시간 알고리즘
N의 거듭제곱 꼴..
#### 지수 시간 알고리즘
매우 느림... 이렇게 오래 걸리는 문제들이 널렸다..

## 시간 복잡도
알고리즘이 실행되는 동안 수행하는 기본적인 연산의 수를 입력의 크기에 대한 함수로 표현.  
- 기본연산
    1. 두 부호 있는 32비트 정수의 사칙연산
    2. 두 실수형 변수의 대소 비교
    3. 한 변수에 다른 변수 대입
- 기본연산이 아닌 것
    - 정수 배열 정렬
    - 두 문자열이 서로 같은지 확인
    - 입력된 수 소인수 분해

### 점근적 시간 표기: O 표기(Big-O Notation)
가장 빨리 증가하는 항만을 남긴 채 나머지는 다 버리고 표기.  
- N에 대한 함수 $f(N)$이 주어질 때, $f(N) = O(g(N))$이라고 쓰는 것의 의미
    - 아주 큰 $N_0$와 $C(N_0, C>0)$를 적절히 선택하면 $N_0\le N$인 모든 $N$에 대해 $|f(N)| \le C \times |g(N)|$이 참이 되도록 할 수 있다.

#### 분할 상환 분석(amortized analysis)
위키: `어떠한 임의의 알고리즘에 대해서, 어떤 연산은 자원적 측면에서 상당한 비용을 소모할 수 있지만, 반면 다른 연산은 그렇게 고비용을 소모하지 않을 수 있다. 분할상환 분석은 알고리즘의 전반적인 연산 집합에 대해 비용이 높은 연산, 그리고 비용이 덜한 연산 모두를 함께 고려하는 기법`

참고 사이트  
https://zeddios.tistory.com/60  
http://blog.naver.com/PostView.nhn?blogId=blackblight&logNo=40098141058&beginTime=0&jumpingVid=&from=search&redirect=Log&widgetTypeCall=true

### 수행시간 어림짐작
**반복문 수행 횟수가 1억($10^8$)을 넘어가면 시간 제한을 초과할 가능성이 있다!!**

#### 최대 연속 부분 구간 합 문제
1차원 배열에서 연속된 부분 구간 중 그 합이 최대인 구간을 찾는 문제.  
시간복잡도가 다른 여러 풀이를 살펴보자.  

1. 모든 부분 구간을 순회하면서 그 합을 계산.
    - $N^2$개의 후보 구간을 검사하고 각 구간의 합을 구하는 데 N번. 총 시간 복잡도 $O(N^3)$
    ```c++
    const int MIN = numericlimits<int>::min();
    
    int inefficientMaxSum(const vector<int>& A) {
        int N = A.size(), ret = MIN;
        for (int i = 0; i < N; i++) 
            for (int j = i; j < N; j++) {
                int sum = 0;
                for (int k = i; k <= j; k++)
                    sum += A[k];
                ret = max(ret, sum);
            }
        return ret;
    }
    ```
    

2. 이동평균 빠르게 구하는 변환을 이용해 $O(N^2)$으로 개선..
    ```c++
    int betterMaxSum(const vector<int>& A) {
        int N = A.size(), ret = MIN;
        for (int i = 0; i < N; i++) {
            int sum = 0;
            for (int j = i; j < N; j++) {
                sum += A[j];
                ret = max(ret, sum);
            }
        }
        return ret;
    }
    ```

3. 분할 정복 기법을 이용하여 $O(NlogN)$ 구현.
    - 배열을 절반으로 자르면 우리가 원하는 부분은 어느 하나에 속하거나 걸쳐 있을 수 있다.
    - 각 경우의 답을 재귀 호출과 탐욕법을 이용해 계산.
    ```c++
    int fastMaxSum(const vector<int>& A, int lo, int hi) {
        // 기저 사례: 구간의 길이가 1인 경우
        if (lo == hi) return A[lo];
        // 배열을 A[lo..mid], A[mid+1..hi] 두 조각으로 나눈다.
        int mid = (lo + hi) / 2;
        // 1. 두 부분에 모두 걸쳐 있는 최대 합 구간을 찾는다. 
        // A[i..mid]와 A[mid+1..j] 형태를 갖는 구간의 합..
        // A[i..mid] 형태를 갖는 최대 구간을 찾는다.
        int left = MIN, right = MIN, sum = 0;
        for (int i = mid; i >= lo; i--) {
            sum += A[i];
            left = max(left, sum);
        }
        // A[mid+1..j]형태를 갖는 최대 구간을 찾는다.
        sum = 0;
        for (int j = mid+1; j <= hi; j++) {
            sum += A[j];
            right = max(right, sum);
        }
        // 2. 최대 구간이 두 조각 중 하나에만 속해 있는 경우의 답을 재귀 호출로 찾는다.
        int single = max(fastMaxSum(A, lo, mid), fastMaxSum(A, mid+1, hi));

        // 두 경우 중 최대치를 반환.
        return max(left + right, single);
    }
    ```

4. 동적 계획법을 사용하여 선형 시간에 풀기.
    - A[i]를 오른쪽 끝으로 갖는 구간의 최대 합을 반환하는 함수 maxAt(i)를 정의.
    - A[i]에서 끝나는 최대 합 부분 구간은 항상 
        1. `A[i] 하나만으로 구성돼있거나`
        2. `A[i-1]를 오른쪽 끝으로 갖는 최대 합 부분 구간의 오른쪽에 A[i]를 붙인 형태`
        로 구성돼있다.
    - maxAt(i) = max(0, maxAt(i - 1)) + A[i]
    ```c++
    int fastestMaxSum(const vector<int>& A) {
        int N = A.size(), ret = MIN, psum = 0;
        for (int i = 0; i < N; i++) {
            psum = max(psum, 0) + A[i];
            ret = max(psum, ret);
        }
        return ret;
    }
    ```

### 계산 복잡도 클래스: P, NP, NP-완비
문제의 '어려움'이란 '해당 문제를 해결하는 빠른 알고리즘이 있느냐'를 나타냄.

1. 빠른 알고리즘
    - 다항 시간 알고리즘이나 그보다 빠른 알고리즘들.
    - P 문제라고 부른다.
2. 어려운 문제
    - NP문제
        - 다항 시간 알고리즘이 존재하지 않는 문제들이란 뜻이 아니다!!
        - 어떤 문제를 다항 시간에 풀 수 있음을 증명하긴 쉽지만 풀 수 없음을 보이기는 어렵다...
        - **답이 주어졌을 때 이것이 정답인지를 다항 시간 내에 확인할 수 있는 문제**라는 뜻.
        - 모든 P문제들은 NP문제에도 포함됨.
    - 환산(reduction)기법을 통해 두 문제의 난이도를 비교할 수 있다.
    - SAT(Satisfiability Problem)문제는 모든 NP문제 이상으로 어렵다!!
        - N개의 불린 값 변수로 구성된 논리식을 참으로 만드는 변수 값들의 조합을 찾는 문제.
    - NP문제보다 어려운 문제들의 집합을 NP-난해(NP-Hard) 문제라고 부른다.
    - NP-난해면서 NP인 문제들을 NP-완비(NP-Complete)문제라고 한다.
        - 대표적인 예로 부분 집합 합 문제..
3. P=NP?
    - 밀레니엄 7대 수학 난제로 P와 NP가 같은지 확인하는 문제.
    - NP-난해 문제 중 하나를 다항 시간에 풀 수 있다면 이 알고리즘을 이용해 NP에 속한 모든 문제를 다항 시간에 풀 수 있다.
        - 이 경우 P = NP가 된다.
    - 반대로 NP문제 중 하나를 골라 P에 포함되지 않음을 증명하면 P != NP 임을 보일 수 있다.
    - 어려운 문제들은 모델링 하거나 근사해 이용..
    

- **마스터 정리(Master Theorem)**
    - 어떤 함수의 수행시간이 특정 형태의 함수로 표현될 때 이 함수의 O 표기법을 쉽게 알 수 있다.